In [ ]:
!java -version


openjdk version "11.0.25" 2024-10-15
OpenJDK Runtime Environment (build 11.0.25+9-post-Ubuntu-1ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.25+9-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)


In [ ]:
# Download Spark
!wget -q https://downloads.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

# Extract Spark
!tar -xvf spark-3.5.0-bin-hadoop3.tgz


tar: spark-3.5.0-bin-hadoop3.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [ ]:
!rm -f spark-3.5.0-bin-hadoop3.tgz
!wget -q https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!ls -l



total 391016
drwxr-xr-x 1 root root      4096 Jan 31 14:18 sample_data
-rw-r--r-- 1 root root 400395283 Sep  9  2023 spark-3.5.0-bin-hadoop3.tgz


In [ ]:
!tar -xvf spark-3.5.0-bin-hadoop3.tgz


spark-3.5.0-bin-hadoop3/
spark-3.5.0-bin-hadoop3/kubernetes/
spark-3.5.0-bin-hadoop3/kubernetes/tests/
spark-3.5.0-bin-hadoop3/kubernetes/tests/pyfiles.py
spark-3.5.0-bin-hadoop3/kubernetes/tests/decommissioning.py
spark-3.5.0-bin-hadoop3/kubernetes/tests/autoscale.py
spark-3.5.0-bin-hadoop3/kubernetes/tests/python_executable_check.py
spark-3.5.0-bin-hadoop3/kubernetes/tests/worker_memory_check.py
spark-3.5.0-bin-hadoop3/kubernetes/tests/py_container_checks.py
spark-3.5.0-bin-hadoop3/kubernetes/tests/decommissioning_cleanup.py
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/spark/
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/spark/decom.sh
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/spark/bindings/
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/spark/bindings/R/
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/spark/bindings/R/Dockerfile
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/spark/bindings/python/
spark-3.5.0-bin-hadoop3/kubern

In [ ]:
!ls -l


total 391020
drwxr-xr-x  1 root root      4096 Jan 31 14:18 sample_data
drwxr-xr-x 13 1000 1000      4096 Sep  9  2023 spark-3.5.0-bin-hadoop3
-rw-r--r--  1 root root 400395283 Sep  9  2023 spark-3.5.0-bin-hadoop3.tgz


In [ ]:
import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

findspark.init()


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("TestSession").getOrCreate()

# Display Spark session info
spark


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving healthcare_dataset.csv to healthcare_dataset (1).csv


In [ ]:
df = spark.read.csv("healthcare_dataset.csv", header=True, inferSchema=True)
df.show()


+-------------------+---+------+----------+-----------------+-----------------+----------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+
|               name|age|gender|blood_type|medical_condition|date_of_admission|          doctor|            hospital|insurance_provider|    billing_amount|room_number|admission_type|discharge_date| medication|test_results|
+-------------------+---+------+----------+-----------------+-----------------+----------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+
|      Bobby JacksOn| 30|  Male|        B-|           Cancer|       2024-01-31|   Matthew Smith|     Sons and Miller|        Blue Cross|18856.281305978155|        328|        Urgent|    2024-02-02|Paracetamol|      Normal|
|       LesLie TErRy| 62|  Male|        A+|          Obesity|       2019-08-20| Samantha Davies|            

In [ ]:
# Drop rows with missing values
df_cleaned = df.na.drop()

# Or fill missing values (e.g., if 'blood_type' is missing, fill it with 'Unknown')
df_cleaned = df.fillna({'blood_type': 'Unknown', 'medication': 'None'})


In [ ]:
df_cleaned = df_cleaned.dropDuplicates()


In [ ]:
from pyspark.sql.functions import when, col

# Categorize age into different groups (child, adult, senior)
df_cleaned = df_cleaned.withColumn('age_group',
                                  when(col('age') < 18, 'child')
                                  .when((col('age') >= 18) & (col('age') < 60), 'adult')
                                  .otherwise('senior'))


In [ ]:
df_filtered = df_cleaned.filter(df_cleaned['age'] > 50)
df_filtered.show()


+------------------+---+------+----------+-----------------+-----------------+--------------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+---------+
|              name|age|gender|blood_type|medical_condition|date_of_admission|              doctor|            hospital|insurance_provider|    billing_amount|room_number|admission_type|discharge_date| medication|test_results|age_group|
+------------------+---+------+----------+-----------------+-----------------+--------------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+---------+
|        JOhN eLlis| 83|Female|        A-|     Hypertension|       2023-06-15|        Thomas Bruce|        Moore-Waters|  UnitedHealthcare|14371.539698666105|        341|        Urgent|    2023-06-30|Paracetamol|Inconclusive|   senior|
|    JUlIe mattHEWS| 85|Female|        O+|          Obes

In [ ]:
from pyspark.sql.functions import to_date

df_cleaned = df_cleaned.withColumn('date_of_admission', to_date(col('date_of_admission'), 'yyyy-MM-dd'))
df_cleaned = df_cleaned.withColumn('discharge_date', to_date(col('discharge_date'), 'yyyy-MM-dd'))


In [ ]:
df_grouped = df_cleaned.groupBy('hospital').agg({'billing_amount': 'sum'})
df_grouped.show()


+--------------------+-------------------+
|            hospital|sum(billing_amount)|
+--------------------+-------------------+
|       Group Edwards| 142763.55528154678|
|        Calderon LLC|  42140.21735537132|
|         Johns-Evans|  39674.23992427462|
|          PLC Weaver|   80320.3342229739|
|        PLC Browning| 27191.235288997206|
|        Morris-Brady|  20291.35934434915|
|Holmes Reed and J...|  9492.225487246009|
|Newman Bowers and...| 43681.441937410746|
|        Aguilar-Bass|  14896.97500213013|
|Davis, Reed Young...| 13430.292173016689|
|          LLC Massey|   71446.9940966842|
|Parks Conner, and...|  38980.29230333072|
|Blackwell Martine...| 30898.651639794058|
|Rogers, Wiley and...|   81440.8993154909|
|Rogers, Flores an...| 3187.6198962730923|
|           Smith PLC| 1029424.4491163138|
|and Palmer Castro...| 19905.414373307838|
|           Mills Ltd|  57276.84152939514|
|       Dixon-Jenkins| 10513.192365805622|
|    Caldwell-Swanson| 16655.049095004768|
+----------

In [ ]:
df_cleaned = df_cleaned.withColumn('gender', when(col('gender') == 'M', 1).otherwise(0))

In [ ]:
df_condition_filtered = df_cleaned.filter(df_cleaned['medical_condition'] == 'Diabetes')
df_condition_filtered.show()


+-------------------+---+------+----------+-----------------+-----------------+--------------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+---------+
|               name|age|gender|blood_type|medical_condition|date_of_admission|              doctor|            hospital|insurance_provider|    billing_amount|room_number|admission_type|discharge_date| medication|test_results|age_group|
+-------------------+---+------+----------+-----------------+-----------------+--------------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+---------+
|        wAYNE hODGE| 47|     0|        A-|         Diabetes|       2019-10-09|      Nicole Manning|     Sons Flores and|        Blue Cross| 9594.886274485909|        312|     Emergency|    2019-11-07|Paracetamol|      Normal|    adult|
|        jOsHua HIll| 52|     0|        B-|         

In [ ]:
df_avg_billing = df_cleaned.groupBy('room_number').agg({'billing_amount': 'avg'})
df_avg_billing.show()

+-----------+-------------------+
|room_number|avg(billing_amount)|
+-----------+-------------------+
|        496| 25347.033544068487|
|        471|  25119.00951561819|
|        148|   26362.3659431157|
|        463|  24746.00683004974|
|        243| 26655.239306883093|
|        392| 26769.188991887615|
|        451|  25370.34409933882|
|        251|  24998.98577763952|
|        137| 25234.767843571317|
|        458|  24916.97003649313|
|        255|  24605.68828803591|
|        481| 26820.377038606053|
|        472| 26423.762637750522|
|        296|  23693.58733148496|
|        133|  25236.08281534377|
|        322| 25198.047827320548|
|        362|  24953.20250737971|
|        321| 24644.920190766858|
|        375|   26603.6885963109|
|        155| 24750.956977228307|
+-----------+-------------------+
only showing top 20 rows



In [ ]:
!pip install dask
!pip install dask[dataframe]


INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 14.8 MB/s eta 0:00:00


In [ ]:
import dask.dataframe as dd


In [ ]:
# Load data using Dask
df_dask = dd.read_csv('healthcare_dataset.csv')

# View the first few rows to check if it's loaded correctly
df_dask.head()  # Note: Dask doesn't load the entire data immediately, this will show a preview.


,name,age,gender,blood_type,medical_condition,date_of_admission,doctor,hospital,insurance_provider,billing_amount,room_number,admission_type,discharge_date,medication,test_results
0,Bobby JacksOn,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal
1,LesLie TErRy,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive
2,DaNnY sMitH,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal
3,andrEw waTtS,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,"Hernandez Rogers and Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal
4,adrIENNE bEll,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal


In [ ]:
# Drop rows with missing values
df_dask_cleaned = df_dask.dropna()

# Or, fill missing values in specific columns
df_dask_cleaned = df_dask.fillna({'blood_type': 'Unknown', 'medication': 'None'})

# Remove duplicates
df_dask_cleaned = df_dask_cleaned.drop_duplicates()

# Check the first few rows of the cleaned data
df_dask_cleaned.head()


,name,age,gender,blood_type,medical_condition,date_of_admission,doctor,hospital,insurance_provider,billing_amount,room_number,admission_type,discharge_date,medication,test_results
0,Bobby JacksOn,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal
1,LesLie TErRy,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive
2,DaNnY sMitH,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal
3,andrEw waTtS,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,"Hernandez Rogers and Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal
4,adrIENNE bEll,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal


In [ ]:
import dask.dataframe as dd

# Ensure the 'age' column is numeric
df_dask_cleaned['age'] = df_dask_cleaned['age'].astype(float)

# Create 'age_group' column
df_dask_cleaned['age_group'] = df_dask_cleaned['age'].map_partitions(
    lambda x: x.apply(lambda age: 'child' if age < 18 else 'adult' if age < 60 else 'senior'),
    meta=('age_group', 'str')  # Correct meta definition
)

# Check the new column
df_dask_cleaned.head()


,name,age,gender,blood_type,medical_condition,date_of_admission,doctor,hospital,insurance_provider,billing_amount,room_number,admission_type,discharge_date,medication,test_results,age_group
0,Bobby JacksOn,30.0,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal,adult
1,LesLie TErRy,62.0,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive,senior
2,DaNnY sMitH,76.0,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal,senior
3,andrEw waTtS,28.0,Female,O+,Diabetes,2020-11-18,Kevin Wells,"Hernandez Rogers and Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal,adult
4,adrIENNE bEll,43.0,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal,adult


In [ ]:
# Group by 'hospital' and calculate total billing amount
df_dask_agg = df_dask_cleaned.groupby('hospital')['billing_amount'].sum().compute()

# Show the aggregated result
print(df_dask_agg)


hospital
Abbott Inc                       38052.041917
Abbott Ltd                       29877.586483
Abbott Moore and Williams,       24799.596339
Abbott and Thompson, Sullivan    16738.569765
Abbott, Peters and Hoffman       37684.793727
                                     ...     
and Zimmerman Sons               32706.652625
and Zuniga Davis Carlson,        42867.041298
and Zuniga Francis Peterson,     33689.630726
and Zuniga Sons                  33950.170483
and Zuniga Thompson, Blake       22067.428763
Name: billing_amount, Length: 39876, dtype: float64


In [ ]:
# Save processed data as CSV
df_dask_cleaned.to_csv('cleaned_healthcare_dataset.csv', index=False, single_file=True)


['/content/cleaned_healthcare_dataset.csv']

In [ ]:
df_dask_cleaned.compute().to_csv('/content/processed_healthcare_data.csv', index=False)


In [ ]:
from google.colab import files
files.download('/content/processed_healthcare_data.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>